## SDAD Broadbandnow.com Scraper Code
### Last Edit: 2023-01-12

Requires addresses from Corelogic/similar source at the block level with 2 important columns: geoid_blk and mail_address. Original approach uses one address per block group and first generates cleaned list of addresses and block groups from this. Then, proceeds to scrape all package information for those selected addresses. Reports results at block group, tract, and county level within dataframe, which can be exported.

In [377]:
# imports
# generic imports
import numpy as np
import re
import os
import time
import pandas as pd
import random
from bs4 import BeautifulSoup
from tqdm import tqdm
from slugify import slugify
import math

# selenium imports
import selenium
from selenium import webdriver
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [378]:
# check if xpath exists, if not return false
def check_exists_by_xpath(driver, xpath):
    '''
    Description:
        Check existence of xpath on page
    
    Inputs:
        webdriver: your webdriver
        xpath: whatever element we are looking for
        
    Outputs:
        returns True if xpath exists, False if not
    '''
    # try to find element
    try:
        # driver.find_element_by_xpath(xpath)
        driver.find_element("xpath", xpath)
    
    # throw exception and return false if unable to find
    except NoSuchElementException:
        return False
    return True

In [379]:
# IMPORTANT: Corelogic provides BLOCK level FIPs code, but we use BLOCK GROUP level data here
def read_and_clean_addresses_for_bgs(data, need_subset = True, size_subset = 3):
    '''
    Description:
        Check existence of xpath on page
    
    Inputs:
        data: string, name of csv you want to use (includes .csv ending)
        need_subset: boolean, True if using subset of data (originally used 1st address within each bg from list of 3) - default = True
        size_subset: integer, if subsetting, selects every "nth" row (not necessary to mess with this param if using 1 address per bg) - default = 3
        
    Outputs:
        returns True if xpath exists, False if not
    '''
    # read in csv, drop index, and update block column
    address_sample_3_per_bg = pd.read_csv(data, index_col = 0)
    address_sample_3_per_bg = address_sample_3_per_bg.reset_index(drop = True)
    address_sample_3_per_bg['geoid_blk'] = address_sample_3_per_bg.geoid_blk.astype(str)
    
    # drop lat 4 digits of mail address to get short zipcode
    a = address_sample_3_per_bg.mail_address.values
    a = np.array([a[i][0:-4] if a[i][-9].isdigit() else a[i] for i in range(len(a))])
    
    # get block group geoid
    address_sample_3_per_bg['geoid_bg'] = address_sample_3_per_bg.geoid_blk.str.slice(start=0, stop=12)
    
    # if data needs subsetting (I had 3 addresses )
    if need_subset:
        addresses = a[::size_subset]
        block_geoids = address_sample_3_per_bg.geoid_bg[::size_subset]
        
    else:
        addresses = a
        block_geoid = address_sample_3_per_bg.geoid_bg
    
    return addresses, block_geoids.values

In [380]:
def get_empty_df(address):
    plan_df = pd.DataFrame()
    plan_df['address'] = address
    plan_df['speed'] = None
    plan_df['down_up'] = None
    plan_df['price'] = None
    plan_df['name'] = None
    plan_df['type'] = None
    plan_df['address'] = None        
    plan_df['success'] = False
    return plan_df

In [381]:
def search_address2(address, driver, driver_wait = 20):
    '''
    Description:
        Check existence of xpath on page
    
    Inputs:
        address: string, single home address we are scraping for
        driver: your webdriver
        driver_wait: integer, wait time for driver - default = 20
        
    Outputs:
        returns True if xpath exists, False if not
    '''
    # wait until search bar is clickable and enter address
    wait = WebDriverWait(driver, driver_wait)
    search = wait.until(EC.element_to_be_clickable((By.ID, 'plan-search')))
    search.clear()
    search.send_keys("{}".format(address))

    # sleep, then go to top suggested address
    time.sleep(sleep_time)
    go_top = check_exists_by_xpath(driver, '//*[@id="plans-search"]/div/div/div[1]/div/div/div/ul')

    # click top address
    if go_top:
        go_top_address = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="plans-search"]/div/div/div[1]/div/div/div/ul/li')))
        go_top_address.click()
        
    return go_top

def extract_page(address, driver):
    # bs - scrape page
    html = driver.page_source
    soup = BeautifulSoup(html)

    errors = soup.find_all(attrs = {"class": 'c-plans-search-error__heading'})
    
    if len(errors) > 0: # Return error address data frame if getting blocked
        return get_empty_df(address)
    
    plan_dfs = []
    # Iterate through each service plan:
    for plan in soup.find_all(attrs = {"class": 'l-providers-list__item'}):
        # Create empty data frame
        plan_df = pd.DataFrame()
        speed = [s.getText() for s in plan.find_all(attrs = {"class": "c-provider-card__speeds-value"})]
        down_up = [s.getText() for s in plan.find_all(attrs = {"class": "c-provider-card__speeds-label"})]
        price = plan.find(attrs = {"class": "c-provider-card__plan-value"}).getText().split("$")[-1]
        name = plan.find(attrs = {"class": "c-provider-card__provider-name"}).getText().split(". ")[1]
        internet_type = plan.find(attrs = {"class": "c-provider-card__label"}).getText().strip()
        
        plan_df['speed'] = speed
        plan_df['down_up'] = down_up
        plan_df['price'] = price
        plan_df['name'] = name
        plan_df['type'] = internet_type
        plan_df['address'] = address
        plan_df['success'] = True
        
        if plan_df is not None and not plan_df.empty:
            plan_dfs.append(plan_df)
            
    df = pd.concat(plan_dfs)
    # print(df)
    return df

In [382]:
def scrape_prices(driver, addresses, min_wait = 10, max_wait =30, save_folder ='../../data/temp/'):
    '''
    Description:
        Scrape internet packages from Broadbandnow.com - takes each address and scrapes all packages for top match
    
    Inputs:
        driver: your webdriver
        addresses: array of strings, home addresses we are scraping for (first output of read_and_clean_addresses_for_bgs)
        
    Outputs:
        df: a data frame containing the columns:
            address: corresponding address of the package
            price: price of the package
            name: name of the package
            type: type of package?
            speed: speed of the package
            
    '''
    adfs = []
    empty = []
    
    # loop over block group addressed
    for address in tqdm(addresses):
        # try below and exception IF takes too long (increments a counter before skipping address eventually)
        
        address_name = slugify(address)
        if os.path.isfile(save_folder + address_name +'.csv'):            
            continue
            
        try:
            # reload page to clear results (noticed that we run into issues if we do not clear)
            driver.get("https://broadbandnow.com/compare/plans")
            go_top = search_address2(address, driver)

            # select top address
            if not go_top:
                # print('Skipping: Cannot go to the top address')
                empty.append(address)
                continue # skip to next address
            time.sleep(1)
            unable_to_confirm = check_exists_by_xpath(driver, "/html/body/div[2]/div/div/div[1]/section/section/div/div/div[1]/div/section")

            # if able to confirm and go to top address
            if unable_to_confirm:
                # print('Skipping: Cannot go to the top address')
                empty.append(address)
                continue
                
            time.sleep(1)
            load_more = check_exists_by_xpath(driver, '//*[@id="cityPlansListing"]/section/div/div[2]/div/div/section')

            #if load more is an option, then load all packages
            if load_more:
                # load all plans
                load_all_plans = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cityPlansListing"]/section/div/div[2]/div/div/section')))
                load_all_plans.click()

            adf = extract_page(address, driver)

            if not adf.empty:
                # adfs.append(adf)
                adf.to_csv(save_folder + address_name +'.csv', index=False)
            
            # Be respectful of pinging the server
            time.sleep(random.randint(min_wait, max_wait))
            
            # select edit option to change address
            edit = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="plans-search"]/div/div/div/h1/span')))
            edit.click()

        # if try fails, throw exception and increment counter (retry until problem_counter hits 5)
        # throws error if we try to edit search plans but this is not an option because nothing was searched after hitting home page
        except TimeoutException as ex:
            # DO something
            empty.append(address)

    # close driver
    driver.quit()        

    return empty


In [383]:
# Storing previous column names
#     df = pd.DataFrame({"address": result, "price": flatten(all_prices), "name": flatten(all_names),
#                        "type": flatten(all_type_list), "download": download, "upload": upload,
#                        "block_group": short_blockgroup_geoid2, "tract": short_tract_geoid2,
#                        "county": short_county_geoid2})

### Running the functions above

In [384]:
# start driver
options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Last I checked this was necessary.
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

driver.get("https://broadbandnow.com/compare/plans")



#driver.maximize_window()

# set driver params
driver_wait = 20
sleep_time = 2
wait = WebDriverWait(driver, driver_wait)

/var/folders/31/_flrh_251y578sg72lx1yb780000gn/T/ipykernel_62357/2655166332.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
/var/folders/31/_flrh_251y578sg72lx1yb780000gn/T/ipykernel_62357/2655166332.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [385]:
df = pd.read_csv('../../data/usa_reverse_geocode_county_centroid.csv')
df

,lat,lon,geometry,address_found,place_id,licence,osm_type,osm_id,osm_lat,osm_lon,...,city_district,natural,man_made,office,railway,city_block,industrial,district,historic,geoid
0,31.888596,-100.529870,"c(-100.529870353506, 31.8885959252504)","Wildcat Road, Coke County, Texas, 76945, Unite...",109200510.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,20792382.0,31.887038,-100.528915,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48081
1,27.433807,-97.727303,"c(-97.7273031840018, 27.4338070741923)","Kleberg County, Texas, United States",298339971.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1828394.0,27.398313,-97.748510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48273
2,32.548162,-94.371416,"c(-94.3714157448517, 32.548161686492)","481, North Grove Street, Marshall, Harrison Co...",320460771.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,20952189.0,32.548158,-94.371423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48203
3,33.149522,-95.564008,"c(-95.5640079073386, 33.1495218296386)","Sulphur Springs, Hopkins County, Texas, 75482,...",109580183.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,20978340.0,33.149469,-95.561690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48223
4,32.743567,-101.431712,"c(-101.431711731087, 32.7435672342953)","Willow Valley Road, Gail, Borden County, Texas...",141754701.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,152886870.0,32.744552,-101.428689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,37.237994,-100.366236,"c(-100.366235748342, 37.2379935819825)","Meade County, Kansas, United States",298208302.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1070343.0,37.209112,-100.396302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20119
3216,48.233806,-100.635959,"c(-100.635959163527, 48.2338059362209)","57th Street North, McHenry County, North Dakot...",102298353.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,9700564.0,48.239720,-100.635964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38049
3217,18.346676,-65.813584,"c(-65.8135843704381, 18.3466760900796)","Carretera 9966, Jiménez, Río Grande, Puerto Ri...",109332128.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,22209753.0,18.347216,-65.813402,...,Jiménez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72119
3218,45.771153,-100.051569,"c(-100.051569071343, 45.7711534547604)","112th Street, Campbell County, South Dakota, U...",134360618.0,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,125912099.0,45.767771,-100.051579,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46021


In [386]:
sample = df.sample(3)[['address_found', 'geoid']]

# addresses = list(sample['address_found'])
addresses = list(df['address_found'])
print(len(addresses))

addresses = [v for v in addresses if isinstance(v, str)]
print(len(addresses))

block_geoids = np.array(df['geoid'])
print(len(block_geoids))

3220
3219
3220


#  Probably better to save the files individually before adding together to save progress (and memory)

In [392]:
print(len(os.listdir('../../data/temp/')))

2010


In [396]:
# iterate only through ones that do not exist
# print(slugify(addresses[0]))
# print(os.listdir('../../data/temp/')[0])

addresses = [a for a in addresses if not slugify(a)+'.csv' in os.listdir('../../data/temp/')]
print(len(addresses))

1206


In [387]:
empty = scrape_prices(driver, addresses, 3, 5)
print('Empty: %s' % empty)

  0%|▎                                                                                                                                                                                                                                        | 5/3219 [00:45<8:03:12,  9.02s/it]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.124)
Stacktrace:
0   chromedriver                        0x0000000100fa3b7c chromedriver + 4209532
1   chromedriver                        0x0000000100f2f418 chromedriver + 3732504
2   chromedriver                        0x0000000100be2368 chromedriver + 271208
3   chromedriver                        0x0000000100bbd6fc chromedriver + 120572
4   chromedriver                        0x0000000100c42660 chromedriver + 665184
5   chromedriver                        0x0000000100c54fbc chromedriver + 741308
6   chromedriver                        0x0000000100c1059c chromedriver + 460188
7   chromedriver                        0x0000000100c11644 chromedriver + 464452
8   chromedriver                        0x0000000100f7530c chromedriver + 4018956
9   chromedriver                        0x0000000100f78fd8 chromedriver + 4034520
10  chromedriver  

# Clean and append geoid based on address

In [ ]:
pdf

In [ ]:
pdf['geoid'] = pdf['address'].apply(lambda x: df[df['address_found']==x]['geoid'].values[0])

In [ ]:
pdf

In [ ]:
pdf.to_csv('../../data/broadbandnow_pirces.csv', index=False)